# 🌊 Análise de Passagens de Satélites - Ciclone Akará 🌀

Este notebook demonstra como carregar e visualizar as trajetórias dos satélites durante o evento Akará.

In [ ]:
# Imports
import sys
sys.path.append('../scripts')

import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from pathlib import Path
from datetime import datetime

from config import START_DATE, END_DATE, BBOX, DATASETS

print("✅ Imports concluídos!")

## 📡 Carregar Dados dos Satélites

In [ ]:
def load_satellite_data(data_dir='../data'):
    """Carrega dados de todos os satélites disponíveis"""
    data_path = Path(data_dir)
    satellite_data = {}
    
    print("🔍 Procurando dados de satélites...\n")
    
    for satellite in DATASETS.keys():
        satellite_dir = data_path / satellite.replace('/', '_')
        
        if not satellite_dir.exists():
            print(f"⚠️  {satellite}: diretório não encontrado")
            continue
        
        nc_files = list(satellite_dir.glob('*.nc'))
        
        if not nc_files:
            print(f"⚠️  {satellite}: sem arquivos NetCDF")
            continue
        
        try:
            ds = xr.open_dataset(nc_files[0])
            
            if len(ds.latitude) > 0:
                satellite_data[satellite] = ds
                print(f"✅ {satellite}: {len(ds.latitude)} pontos")
            else:
                ds.close()
                
        except Exception as e:
            print(f"❌ {satellite}: erro ao carregar")
    
    return satellite_data

# Carregar dados
satellite_data = load_satellite_data()
print(f"\n📊 Total: {len(satellite_data)} satélites com dados")

## 📊 Explorar Dados de um Satélite

In [ ]:
# Selecionar primeiro satélite disponível
if satellite_data:
    sat_name = list(satellite_data.keys())[0]
    ds = satellite_data[sat_name]
    
    print(f"🛰️  Explorando: {sat_name}\n")
    print("Variáveis disponíveis:")
    print(list(ds.variables.keys()))
    print("\nDimensões:")
    print(ds.dims)
    print("\nResumo do dataset:")
    display(ds)
else:
    print("❌ Nenhum dado disponível. Execute o download primeiro!")

## 🗺️ Plotar Trajetórias dos Satélites

In [ ]:
# Cores para cada satélite
SATELLITE_COLORS = {
    'CFOSAT': '#00FFFF',
    'CryoSat-2': '#FF1493',
    'HaiYang-2B': '#FFD700',
    'HaiYang-2C': '#FF8C00',
    'Jason-3': '#00FF00',
    'Saral/AltiKa': '#FF0000',
    'Sentinel-3A': '#FFFF00',
    'Sentinel-3B': '#00CED1',
    'Sentinel-6A': '#9370DB',
    'SWOT_nadir': '#FF69B4',
}

# Criar figura
fig = plt.figure(figsize=(14, 10))
ax = plt.axes(projection=ccrs.PlateCarree())

# Limites do mapa
margin = 2.0
ax.set_extent([
    BBOX['west'] - margin,
    BBOX['east'] + margin,
    BBOX['south'] - margin,
    BBOX['north'] + margin
], crs=ccrs.PlateCarree())

# Features geográficos
ax.add_feature(cfeature.LAND, facecolor='lightgray', edgecolor='black', linewidth=0.5)
ax.add_feature(cfeature.OCEAN, facecolor='lightblue', alpha=0.5)
ax.add_feature(cfeature.COASTLINE, linewidth=0.8)
ax.add_feature(cfeature.BORDERS, linewidth=0.5, linestyle='--', alpha=0.7)

# Grid
gl = ax.gridlines(draw_labels=True, linewidth=0.5, color='gray', alpha=0.5, linestyle='--')
gl.top_labels = False
gl.right_labels = False

# Caixa da região de interesse
box_lons = [BBOX['west'], BBOX['east'], BBOX['east'], BBOX['west'], BBOX['west']]
box_lats = [BBOX['south'], BBOX['south'], BBOX['north'], BBOX['north'], BBOX['south']]
ax.plot(box_lons, box_lats, 'k--', linewidth=2, transform=ccrs.PlateCarree(), 
        label='Região de Interesse', alpha=0.8)

# Plotar cada satélite
for satellite, ds in satellite_data.items():
    color = SATELLITE_COLORS.get(satellite, '#FFFFFF')
    
    lons = ds.longitude.values
    lats = ds.latitude.values
    
    # Remover NaNs
    valid_mask = ~(np.isnan(lons) | np.isnan(lats))
    lons = lons[valid_mask]
    lats = lats[valid_mask]
    
    if len(lons) > 0:
        ax.scatter(lons, lats, c=color, s=30, alpha=0.7, 
                  transform=ccrs.PlateCarree(), 
                  edgecolors='black', linewidth=0.5,
                  label=satellite, zorder=5)

# Título e legenda
plt.title(f'🛰️ Passagens de Satélites - Ciclone Akará 🌀\n{START_DATE} a {END_DATE}', 
          fontsize=16, fontweight='bold', pad=20)
ax.legend(loc='upper left', bbox_to_anchor=(1.02, 1), 
         fontsize=10, framealpha=0.9, 
         title='Satélites', title_fontsize=11)

plt.tight_layout()
plt.show()

print("\n✅ Figura gerada!")

## 📈 Estatísticas de Altura de Ondas

In [ ]:
# Coletar estatísticas de altura de ondas
stats = {}

for satellite, ds in satellite_data.items():
    if 'VAVH' in ds.variables:
        wave_heights = ds.VAVH.values
        wave_heights = wave_heights[~np.isnan(wave_heights)]
        
        if len(wave_heights) > 0:
            stats[satellite] = {
                'mean': np.mean(wave_heights),
                'max': np.max(wave_heights),
                'min': np.min(wave_heights),
                'std': np.std(wave_heights),
                'n_points': len(wave_heights)
            }

# Exibir estatísticas
if stats:
    print("📊 Estatísticas de Altura Significativa de Onda (m)\n")
    print(f"{'Satélite':<15} {'N_Pontos':<10} {'Média':<8} {'Máx':<8} {'Mín':<8} {'Desvio'}")
    print("-" * 70)
    
    for sat, st in stats.items():
        print(f"{sat:<15} {st['n_points']:<10} {st['mean']:<8.2f} {st['max']:<8.2f} {st['min']:<8.2f} {st['std']:.2f}")
else:
    print("❌ Nenhuma estatística disponível")

## 🎨 Gráfico de Distribuição de Altura de Ondas

In [ ]:
# Histograma de altura de ondas por satélite
fig, ax = plt.subplots(figsize=(12, 6))

for satellite, ds in satellite_data.items():
    if 'VAVH' in ds.variables:
        wave_heights = ds.VAVH.values
        wave_heights = wave_heights[~np.isnan(wave_heights)]
        
        if len(wave_heights) > 0:
            color = SATELLITE_COLORS.get(satellite, '#FFFFFF')
            ax.hist(wave_heights, bins=30, alpha=0.5, label=satellite, 
                   color=color, edgecolor='black', linewidth=0.5)

ax.set_xlabel('Altura Significativa de Onda (m)', fontsize=12, fontweight='bold')
ax.set_ylabel('Frequência', fontsize=12, fontweight='bold')
ax.set_title('Distribuição de Altura de Ondas por Satélite', fontsize=14, fontweight='bold')
ax.legend(loc='best', fontsize=9)
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("✅ Histograma gerado!")

## 💾 Salvar Figuras

In [ ]:
# Salvar as figuras geradas
# (Re-execute as células de plotagem acima se necessário)

output_dir = Path('../figures')
output_dir.mkdir(exist_ok=True)

print("💾 Para salvar as figuras, execute as células de plotagem")
print("   e adicione plt.savefig() antes de plt.show()")
print(f"\n📁 Diretório de saída: {output_dir.absolute()}")

## 🧹 Cleanup

In [ ]:
# Fechar datasets
for ds in satellite_data.values():
    ds.close()

print("✅ Datasets fechados!")